# IEMAP TUTORIAL
**Python example code to use as a walkthrough in using iemap API** <br/> 
<img src="https://iemap-api.readthedocs.io/en/latest/_static/iemap_logo_nobg.png"></img>
### Full documentation is available [here](https://iemap-api.readthedocs.io/en/latest/index.html)
> Note. API and its documentation is still in development. For any issue please report at ... 

## Important notes before starting
- IEMAP API are implemented by a REST API architecture build-up by some endpoints that require authentication
- **Prior to using IEMAP API users need to register [here](https://ai4mat.enea.it/dashboard/)**
- It is possible to upload the project's metadata and files using the user interface (UI) at the link above or use the python module here described (**account registration is available only using the UI)**
- The UI at the link above is temporary, a more complete and functional interface is in 

>## Install IEMAP Python module using `pip install iemap` 
**Python version >= 3.9 is required**

In [ ]:
# You can install within jupyter notebook using
# if this is a first installation
# ! pip install iemap
# or to upgrade to latest release
# ! pip install --upgrade  iemap

In [ ]:
# import usefull modules
import json
import getpass
from IPython.display import display
import pandas as pd
from pprint import pprint
# import IEMAP module
from iemap import IEMAP

In [ ]:
# To show the installed version of iemap module
!pip show iemap

- ### Step 1 - define account and login 
> **This is a mandatory step**, if you don't have an account yet register [here](https://ai4mat.enea.it/dashboard/).

In [ ]:
# define user credentials
user="user@enea.it"
print(f"\nType password for user: {user}")
# type in your password in a masked input field
pwd=getpass.getpass()
api = IEMAP(user,pwd)
# login 
api.login()

- ### Step 2 - Start defining a new project metadata
> Project metadata have to be provided using a JSON file or python dict with a "valid" schema as defined below
<!-- <img src="schema1.jpg" style="width:600px"/> -->

![schema1](schema1.png)

> #### Note that   
> - Main fields are: projects*,material*, process*, parameters, properties   
> - "project" is build up by "name" and "label", both strings ("name" is required)   
> - "material" is an object with several sub-fields, the only required is "formula"
> - "material" **<sup>1</sup>** has subfields"input" and "output" both having a nested object made up by "cell" (List of List), "lattice" (Object/Dict), "sites" (List of List) and "species" (List) 
> - "process" is an object with fields "method" and "isExperiment" mandatory
> - "parameters" and "properties" are both a List of objects made up by two fields "name"(string)  and "value"(string or numeric)

> ***denotes mandatory fields**
> **<sup>1</sup>** more on this in the example_metadata below

- ### Step 3 - Familiarize with the required JSON schema

In [ ]:
# load json example metadata
# Opening JSON file
f = open("metadata_example.json")
example_metadata = json.load(f)

In [ ]:
pprint(example_metadata)

In [ ]:
# You can check if metadata has a valid schema with the code below
api.metadata=example_metadata
print("Metadata check passed" if  not api.check_metadata() else "Metadata check not passed")

In [ ]:
# Check metadata again, this time removing a required field as project name
example_metadata['project']['name']=None
print("Metadata check passed" if  not api.check_metadata() else "Metadata check not passed")

In [ ]:
# reset metadata before going on as it will be provided using a JSON file
api.metadata=None

- ### Step 4 - Save a new project 
> Having defined project metadata from a JSON file or as Python dict, to save it on DB use `.save` method.   
> After succesfully saved a new project, the API return an ObjectID that uniquely define the saved project, this id can be retrieved later on using `.get_id()` method.   
> No id is defined if not explicetly set with `.set_id()` or if no new project has been already saved.

In [ ]:
# save project metadata as defined in json file
api.save(metadata='./metadata_example.json')

- ### Step 5 - Show current project id 
> **Initially, after login, no id is set**, so you can either:
    1. use `.set_id` and provide a valid id (e.g. from table above)
    2. use `.save`, this will save a new project with its metadata and, evenutally, its files. This will set the project as the current one with the id returned from DB.  
    <br>
    <br> 
    
    **To add project files to a previosuly saved project use `save_project_files`**

In [ ]:
# Show current project id
api.get_id()

### - Step 6 -Show user projects
> Note, below are only example (random) data

In [ ]:
# show user saved projects
api.my_projects()

#### List projects in tabular format (Pandas DataFrame)

In [ ]:
df_proj=pd.DataFrame(api.my_projects())
df_proj

In [ ]:
# grab a valid project id from previous printed list and set one
id_first_proj=df_proj.iloc[0,0]
api.set_id(id_first_proj)

In [ ]:
# verify new id was correctly set
api.get_id()

- ### Step 7 - Add project file to existing project
> For each user project is possible to define one or more file to upload on Server.     
> Note: **If the file to upload is already present on server then it is not actually over written (on file system) but it is added as field on database.<br/>A note stating "(File already present on File System)" in this case is diplayed.**   
> **Only some file types are allowed as: PDF,CSV, XLS, XLSX, TXT, CIF or DOC.**

In [ ]:
# add project file to a previously save project
_= api.save_project_files(project_files=["../migration/BANDs/10_bandsdown.dat"], show_debug_info=True)

- ### Step 8 - Define a new project metadata (with its associated files in any)
> You can save metadata and files in a unique step using `.save` and providing both ***metadata*** and ***list_proj_files***

In [ ]:
# define a list of valid files path
proj_files = ["../migration/CIFs/10.cif", "../migration/BANDs/1_bandsdown.dat"]

# save project and file
api.save(metadata="./metadata_example.json", list_proj_files=proj_files, show_debug_info=True)

- ### Step 9 - Query a specific Project id
> **All project and its id are available as showed at step 6**

In [ ]:
id_last_proj=df_proj.iloc[-1,0]
data=api.query(id_last_proj)
data

- ### Step 10 - Show ONLY files for a specific project

In [ ]:
# project files
if "files" in data[0]:
    files_df=pd.json_normalize(data, record_path='files')
    display(files_df)
else:
    print(f"No files associated to project {data[0]['iemap_id']}")

- ### Step 11  - Add project files to current project 
(or define another project id and add files to this new project id)

In [ ]:
additional_files_proj = [
    "../migration/BANDs/1_bandsup.dat",
    "../migration/BANDs/7_bandsup.dat",
    "../migration/BANDs/1_bandsdown.dat",
    "../migration/QE/0.out",
    "../migration/QE/0.in",
    "../migration/CIFs/10.cif",
]

# save additional project files to current ID
_ =api.save_project_files(project_files=additional_files_proj)

### - Step 12 - Associate a file to a specific property of material inside a project
> For each project is possible to associate one or more file related to the project as a whole,   
but is also possible to upload a file associtaed to a specific property of the material used in the project   
using `.save_property_files` and provinding as argument the property name and its related fiel as below

In [ ]:
# First Query a specific project

result=api.query(id_first_proj)
if result != []:
    display(result[0]["properties"])

In [ ]:
# set the project id and the property to which associate a file
api.set_id(id_first_proj)
api.save_property_files({"absolute_magnetization": "../migration/CIFs/0.cif"}, show_debug_info=True)

In [ ]:
# CHECK IF FILE HAS BEEN CORRECTLY ADDED
dataExistingProj=api.query(id_first_proj)

In [ ]:
if "files" in dataExistingProj[0]:
    files_df=pd.json_normalize(dataExistingProj, record_path='files')
    display(files_df)
else:
    print(f"No files associated to project {data[0]['iemap_id']}")

### - Step 10 - Download a project (property) file
> Knowing the full file name = hash.ext is possible to download the file using `.download_file` (uploaded by the logged in user or not)

In [ ]:
file_hash=files_df.iloc[0,0]
file_ext=str(files_df.iloc[0,1]).split(".")[-1]
full_file_hash=file_hash +"."+file_ext
print(full_file_hash)

In [ ]:
local_path_downloade_file='./downloaded_file.dat'
api.download_file(full_file_hash,local_path_downloade_file)

### - Step 11 - Delete a file from project
> To delete a file from a project use `delete_project_file` method with argument the hash (only) of the file to be removed   
> Note: **You can delete from File System only files you uploaded and if not used in any other project**

In [ ]:
api.delete_project_file("54b56034096881366c52743ca922ad8194710830")